In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import time
import random
import math
import tensorflow as tf
%matplotlib inline

from skmultiflow.drift_detection import PageHinkley, ADWIN
from skmultiflow.data import DataStream
from keras.models import Sequential
from keras.layers import Dense , Dropout , Lambda, Flatten
from tensorflow.keras.optimizers import Adam ,RMSprop
from sklearn.model_selection import train_test_split
from keras import  backend as K
from keras.utils.np_utils import to_categorical
from keras.utils import to_categorical
from scipy.stats import spearmanr
from scipy.stats import entropy
from tensorflow import keras
from skmultiflow.drift_detection import ADWIN

In [2]:
def get_initial_model(inp, out): #get initial model
    model = tf.keras.Sequential([
        Dense(10, input_dim=inp, kernel_initializer='normal', activation='relu'),
        Dense(out, activation='softmax')
        ])
    #optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [3]:
def node_comparison1(node1, node2, epsilon=0.05): #this function is to see if the nodes are atmost epsilon distance apart
  for x, y in zip(node1,node2):
    #print(x,y)
    if isinstance(x, list):
        if((np.linalg.norm(np.array(x)-np.array(y))/len(x))<=epsilon):
            continue
        else:
            return False
    else:
        if(math.sqrt((x-y)*(x-y))<=epsilon):
            continue
        else:
            return False
  return True

In [4]:
def node_comparison(node1, node2):
  for x, y in zip(node1,node2):
    #print(x,y)
    if isinstance(x, list):
        if(sorted(x)==sorted(y)):
            continue
        else:
            return False
    else:
        if(x==y):
            continue
        else:
            return False
  return True

In [5]:
def layer_comparison(layer1, layer2): #compare same layers in two different DNNs
  for node1 in layer1:
    present=False
    for node2 in layer2:
      if (node_comparison1(node1, node2)):
        present=True
    if present==False:
      return False
  return True

In [6]:
def check_models(Model_weights1, Model_weights2): # compare two deep neural networks based on their weights
  for i in range(0,len(Model_weights1), 2):
    layer1=[]
    layer2=[]
    for j in range(len(Model_weights1[i+1].T)):
      Node1=[]
      Node2=[]
      Node1.append(list(Model_weights1[i].T[j]))
      Node1.append(Model_weights1[i+1][j])
      if (i+2<len(Model_weights1)):
        Node1.append(list(Model_weights1[i+2][j]))
      Node2.append(list(Model_weights2[i].T[j]))
      Node2.append(Model_weights2[i+1][j])
      if (i+2<len(Model_weights2)):
        Node2.append(list(Model_weights2[i+2][j]))
      layer1.append(Node1)
      layer2.append(Node2)
    if (layer_comparison(layer1, layer2)):
      continue
    else:
      return False
  return True

In [7]:
#this function is used to average out all the models in the epsilon range
#the problem is different here than compared with model comparison. Shape not important.
def get_avg_weights(models_weights, inp_shape, out_shape):
    avg_sum=get_initial_model(inp_shape, out_shape).get_weights()
    #print(avg_sum)
    for i in range(0,len(avg_sum),2):
        if (i+2<=len(avg_sum)):
            for j in range(len(avg_sum[i])):
                for k in range(len(avg_sum[i][j])):
                    avg_sum[i][j][k]=0
            for j in range(len(avg_sum[i+1])):
                avg_sum[i+1][j]=0
    #print(avg_sum)
    print(models_weights[0])
    for i in range(len(models_weights)):
        for j in range(0, len(avg_sum),2):
            #print(isinstance(avg_sum[j], np.ndarray))
            #if(isinstance(avg_sum[j][0], np.ndarray)):
            for k in range(len(avg_sum[j])):
                avg_sum[j][k]=[avg_sum[j][k][l]+models_weights[i][j][k][l] for l in range(len(avg_sum[j][k]))]
            #print(isinstance(avg_sum[j], np.ndarray))
            #else: gayab kr diya
            for k in range(len(avg_sum[j+1])):
                avg_sum[j+1][k]=avg_sum[j+1][k]+models_weights[i][j+1][k]
    print("yhn tk")
    mean_size=len(models_weights)
    print(mean_size)
    for i in range(0,len(avg_sum),2):
        if (i+2<=len(avg_sum)):
            for j in range(len(avg_sum[i])):
                #print("yhn tk")
                avg_sum[i][j]=[avg_sum[i][j][k]/mean_size for k in range(len(avg_sum[i][j]))]
            for j in range(len(avg_sum[i+1])):
                avg_sum[i+1][j]=avg_sum[i+1][j]/mean_size
    print("Done")
    return avg_sum

In [8]:
# custom functions for f1, precision and recall

from keras import backend as K
from keras.utils import to_categorical
from sklearn import preprocessing

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [9]:
# A function to check if the two samples are different:
def return_samples(Positive, Negative, data, N):
    datasets=[]
    positiveN=int((Positive.shape[0]/data.shape[0])*N)
    negativeN=int(N-positiveN)
    print(negativeN)
    while (Positive.empty==False and Negative.empty==False):
        df1=Positive.sample(min(positiveN, len(Positive)))
        Positive.drop(df1.index, inplace=True)
        """
        drop_df1=np.random.choice(df1.index,(int)(min(positiveN, math.ceil(len(Positive))/2)), replace=False)
        if len(Positive)<positiveN:
            Positive.drop(df1.index, inplace=True)
        else:
            Positive.drop(drop_df1, inplace=True)
        #print(len(Positive))
        """
        df2=Negative.sample(min(negativeN, len(Negative)))
        Negative.drop(df2.index, inplace=True)
        """
        drop_df2 = np.random.choice(df2.index,(int)(min(negativeN, math.ceil(len(Negative))/2)))
        if len(Negative)<negativeN:
            Negative.drop(df2.index, inplace = True)
        else:
            Negative.drop(drop_df2, inplace=True)
        """
        dataset=df1.append(df2, ignore_index=True)
        dataset.sample(frac = 1)
        dataset.sample(frac = 1)
        dataset.sample(frac = 1)
        datasets.append(dataset)
    print("returned datasets")
    return datasets

In [10]:
#generate sampele which are plausible deniable
#there exists a data lets say data
def generate_samples(data, num_samples, N):
    samples=[]
    intersection=[]
    while num_samples>0:    
        Positive = data[data[target_variable]==0]
        Negative = data[data[target_variable]==1]
        positiveN=int((Positive.shape[0]/data.shape[0])*N)
        negativeN=int(N-positiveN)
        df1=Positive.sample(positiveN)
        #Positive.drop(df1.index, inplace=True)
        df2=Negative.sample(negativeN)
        #Negative.drop(df2.index, inplace=True)
        sample=df1.append(df2, ignore_index=False)
        samples.append(sample)
        num_samples-=1

    intersection=list(set(samples[0].index).intersection(samples[1].index))
    for i in range(2,len(samples)):
        intersection=list(set(samples[i].index).intersection(intersection))
    if intersection:
        rnum = random.randint(0, len(samples)-1)
        samples[rnum].drop(intersection)
        print("some intersection")
    print("finished")
    return samples

In [26]:
#A function to return integrally private model weights with the number of recurrence
def get_IP_models(data, epsilon=0.05, target=2):
    target_variable='income'
    N = int(data.shape[0]*0.25)
    
    samples = generate_samples(data, 50, N)
    """
    df1=Positive.sample(positiveN)
    Positive.drop(df1.index, inplace=True)
    df2=Negative.sample(negativeN)
    Negative.drop(df2.index, inplace=True)
    test_data=df1.append(df2, ignore_index=True)
    test_data=test_data.sample(frac = 1) #This is to shuffel the training and testing data
    test_data=test_data.sample(frac = 1)
    test_data=test_data.sample(frac = 1)
    X_test=test_data.drop(columns=[target_variable])
    y_test=to_categorical(test_data[target_variable])
    """

    # adding dense layer
    initial_model= get_initial_model(data.shape[1]-1, 2)
    Models=[]
    val_acc=[]
    train_acc=[]
    test_acc=[]
    val_loss=[]
    train_loss=[]
    ind=0
    add_weights=[]
    while ind<len(samples):
      """"
      print(positiveN, negativeN)
      df1=Positive.sample(min(positiveN, len(Positive)))
      Positive.drop(df1.index, inplace=True)
      df2=Negative.sample(min(negativeN, len(Negative)))
      Negative.drop(df2.index, inplace=True)
      train_data=df1.append(df2, ignore_index=True)
      train_data=train_data.sample(frac = 1) #shuffel test data 3 times
      train_data=train_data.sample(frac = 1) #shuffel test data 3 times
      train_data=train_data.sample(frac = 1) #shuffel test data 3 times

      #all models have different initialization
      # define the sequential model
      """
      train_data=samples[ind]
      ind=ind+1
      """initial_model = keras.Sequential()

        # adding dense layer
      initial_model.add(Dense(5, input_dim=X_test.shape[1], kernel_initializer='normal', activation='relu'))
      initial_model.add(Dense(10, activation='relu'))
      initial_model.add(Dense(5, activation='relu'))

        # adding dense layer with softmax activation/output layer
      initial_model.add(Dense(2, activation='softmax'))
      #initial_model.summary()"""
      ann_model=get_initial_model(data.shape[1]-1, 2) #same intial weights
      ann_model.set_weights(initial_model.get_weights())
      X_train=train_data.drop(columns=[target_variable])
      #train_data[target_variable]=train_data[target_variable]-1 #only for skin_nonskin dataset
      y_train=to_categorical(train_data[target_variable])
      #print(y_train)
      ann_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[f1_m]) # metrics=['accuracy']
      history = ann_model.fit(X_train, y_train, epochs=10, validation_split=0.2, verbose=1)
      print(history.history['f1_m'][-1])
      #ann_model.set_weights(update_weights(ann_model.get_weights()))
      #pred_test=ann_model.predict(X_test)
      present=False
      for i in range(len(Models)):
        if (check_models(Models[i][0], ann_model.get_weights())):
          print("if any")
          Models[i][1]=Models[i][1]+1
          add_weights[i].append(ann_model.get_weights())
          val_acc[i].append(history.history['val_f1_m'])
          train_acc[i].append(history.history['f1_m'])
          #test_acc[i].append(f1_m(y_test, pred_test).numpy())
          val_loss[i].append(history.history['val_loss'])
          train_loss[i].append(history.history['loss'])
          present=True
          break;
      if present==False:
        add_weights.append([ann_model.get_weights()])
        Models.append([ann_model.get_weights(), 1])
        val_acc.append([history.history['val_f1_m']])
        train_acc.append([history.history['f1_m']])
        #test_acc.append([f1_m(y_test, pred_test).numpy()])
        val_loss.append([history.history['val_loss']])
        train_loss.append([history.history['loss']])
    return add_weights, Models, val_acc, train_acc, train_loss, val_acc, val_loss

In [12]:
#do we need to find all the models or only the mean recommended models?
def epsilon_mean_recommendation(add_weights,data):
    mean_model_weights=[]
    for i in range(len(add_weights)):
        mean_model_weights.append(get_avg_weights(add_weights[i],data.shape[1]-1, 2))
    mean_models=[]
    mean_model_train_metrics=[]
    mean_model_loss=[]
    mean_model_acc=[]
    mean_model_test_metrics=[]
    mean_model_test_loss=[]
    mean_model_test_acc=[]
    y = to_categorical(data[target_variable])
    X = data.drop(columns=target_variable)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    for i in range(len(add_weights)):
        init_model=get_initial_model(X_test.shape[1], 2)
        init_model.set_weights(mean_model_weights[i])
        mean_model_train_metrics.append(init_model.evaluate(X_train, y_train))
        mean_model_loss.append(mean_model_train_metrics[i][0])
        mean_model_acc.append(mean_model_train_metrics[i][1])
        mean_model_test_metrics.append(init_model.evaluate(X_test, y_test))
        mean_model_test_loss.append(mean_model_test_metrics[i][0])
        mean_model_test_acc.append(mean_model_test_metrics[i][1])
    return mean_model_weights, mean_model_acc, mean_model_loss, mean_model_test_acc, mean_model_test_loss

In [13]:
def drift_detection_each_model(models, incoming_data, adwin):
    prediction_probe=[]
    #print(type(incoming_data))
    #print(incoming_data.drop([target_variable]))
    y_test=to_categorical(incoming_data[target_variable])
    print(y_test)
    X_test=incoming_data.drop(columns=target_variable)
    evaluate=[]
    init_model=get_initial_model(incoming_data.shape[1]-1, 2)
    #print(X_test, y_test)
    entropy_list=[[]]*len(models)
    for i in range(len(models)):
        init_model.set_weights(models[i])
        print(init_model.evaluate(X_test, y_test))
        evaluate.append(init_model.evaluate(X_test, y_test))
        #prediction_probe.append(models.predict(X_test))
        entropy_list[i].append(np.average(entropy(init_model.predict(incoming_data.drop(columns=target_variable)), axis=0)))
    print(entropy_list)
    detected=False
    
    for i in range(len(entropy_list)):
        adwin.add_element(entropy_list[i][0])
        if adwin.detected_change():
            detected=True
            break
    return detected, evaluate, adwin

In [14]:
def update_train_data(train_data, incoming_data):
    train_data.drop(index=train_data.index[:len(incoming_data)], inplace=True)
    train_data=train_data.append(incoming_data, ignore_index=True)
    return train_data

In [27]:
from sklearn import preprocessing
#working over adult income dataset
df=pd.read_csv("adult.csv", sep=";", na_values='?')
df = df.dropna(axis = 0)
df = df.reset_index(drop=True)
#target class change

#df['income'] = (df['income']==("<=50K")).astype(int)

print(sum((df['income']==1).astype(int)))

#divide string and integer columns

df_str = df.select_dtypes(include='object')
df_int = df.select_dtypes(exclude='object')
#print(df_int.head())

#handle categorical data
df_str = pd.get_dummies(df_str)

#print(df_str.head())

target = df_int['income']
x = df_int.drop(columns='income')
target_variable='income'
column_names = x.columns.values

scaler = preprocessing.MinMaxScaler()
#scaler.fit(x)
x_stndrd = scaler.fit_transform(x)
type(x_stndrd)

x_stndrd = pd.DataFrame(x_stndrd)
x_stndrd.columns = column_names

data = pd.concat([x_stndrd,df_str, target],axis=1)

11208


In [28]:
#now the basic architecture go for stream 5-10% of the dataset then take 2-5% in each time step
from skmultiflow.data import DataStream

stream = DataStream(data)

two_percent = int(stream.n_remaining_samples()*0.01)
five_percent = int(stream.n_remaining_samples()*0.05)
initial_data = stream.next_sample(int(stream.n_remaining_samples()*0.1))
print(len(initial_data[1]))

4522


In [29]:
data.columns[:-1]

Index(['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'workclass_Federal-gov', 'workclass_Local-gov',
       'workclass_Private', 'workclass_Self-emp-inc',
       ...
       'native-country_Portugal', 'native-country_Puerto-Rico',
       'native-country_Scotland', 'native-country_South',
       'native-country_Taiwan', 'native-country_Thailand',
       'native-country_Trinadad&Tobago', 'native-country_United-States',
       'native-country_Vietnam', 'native-country_Yugoslavia'],
      dtype='object', length=104)

In [30]:
data_init=pd.DataFrame(initial_data[0], columns=data.columns[:-1])
data_init[target_variable]=initial_data[1]
print(data_init)

           age    fnlwgt  educational-num  capital-gain  capital-loss  \
0     0.109589  0.144430         0.400000      0.000000      0.000000   
1     0.287671  0.051677         0.533333      0.000000      0.000000   
2     0.150685  0.219011         0.733333      0.000000      0.000000   
3     0.369863  0.099418         0.600000      0.076881      0.000000   
4     0.232877  0.125398         0.333333      0.000000      0.000000   
...        ...       ...              ...           ...           ...   
4517  0.082192  0.182544         0.533333      0.000000      0.000000   
4518  0.000000  0.075661         0.333333      0.000000      0.000000   
4519  0.410959  0.117689         0.533333      0.000000      0.000000   
4520  0.356164  0.092780         0.600000      0.000000      0.000000   
4521  0.301370  0.012929         0.600000      0.000000      0.340909   

      hours-per-week  workclass_Federal-gov  workclass_Local-gov  \
0           0.397959                    0.0            

In [31]:
#get details of models in the epsilon integrally private range
add_weights, Models, val_acc, train_acc, train_loss, val_acc, val_loss = get_IP_models(data_init, epsilon=0.05, target=2)
Models

finished
Epoch 1/10
29/29 [==============================] - 0s 4ms/step - loss: 0.5268 - f1_m: 0.8384 - val_loss: 1.4228 - val_f1_m: 0.0000e+00
Epoch 2/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2927 - f1_m: 0.9461 - val_loss: 2.1072 - val_f1_m: 0.0000e+00
Epoch 3/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2256 - f1_m: 0.9429 - val_loss: 2.4684 - val_f1_m: 0.0000e+00
Epoch 4/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2063 - f1_m: 0.9461 - val_loss: 2.5845 - val_f1_m: 0.0000e+00
Epoch 5/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1964 - f1_m: 0.9429 - val_loss: 2.5752 - val_f1_m: 0.0000e+00
Epoch 6/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1888 - f1_m: 0.9429 - val_loss: 2.4742 - val_f1_m: 0.0000e+00
Epoch 7/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1817 - f1_m: 0.9461 - val_loss: 2.3982 - val_f1_m: 0.0000e+00
Epoch 8/10
29/29 [=====================

29/29 [==============================] - 0s 1ms/step - loss: 0.1712 - f1_m: 0.9461 - val_loss: 2.1316 - val_f1_m: 0.0000e+00
0.9461206793785095
if any
Epoch 1/10
29/29 [==============================] - 0s 4ms/step - loss: 0.5203 - f1_m: 0.8642 - val_loss: 1.4244 - val_f1_m: 0.0000e+00
Epoch 2/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2904 - f1_m: 0.9461 - val_loss: 2.1126 - val_f1_m: 0.0000e+00
Epoch 3/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2241 - f1_m: 0.9397 - val_loss: 2.4328 - val_f1_m: 0.0000e+00
Epoch 4/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2063 - f1_m: 0.9461 - val_loss: 2.4987 - val_f1_m: 0.0000e+00
Epoch 5/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1960 - f1_m: 0.9429 - val_loss: 2.4736 - val_f1_m: 0.0000e+00
Epoch 6/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1878 - f1_m: 0.9461 - val_loss: 2.3856 - val_f1_m: 0.0000e+00
Epoch 7/10
29/29 [===============

29/29 [==============================] - 0s 1ms/step - loss: 0.1674 - f1_m: 0.9429 - val_loss: 2.3291 - val_f1_m: 0.0000e+00
Epoch 10/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1623 - f1_m: 0.9461 - val_loss: 2.1827 - val_f1_m: 0.0000e+00
0.9461206197738647
if any
Epoch 1/10
29/29 [==============================] - 0s 4ms/step - loss: 0.5198 - f1_m: 0.8438 - val_loss: 1.4512 - val_f1_m: 0.0000e+00
Epoch 2/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2895 - f1_m: 0.9461 - val_loss: 2.1154 - val_f1_m: 0.0000e+00
Epoch 3/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2273 - f1_m: 0.9461 - val_loss: 2.4772 - val_f1_m: 0.0000e+00
Epoch 4/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2070 - f1_m: 0.9429 - val_loss: 2.5610 - val_f1_m: 0.0000e+00
Epoch 5/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1965 - f1_m: 0.9461 - val_loss: 2.5284 - val_f1_m: 0.0000e+00
Epoch 6/10
29/29 [==============

29/29 [==============================] - 0s 1ms/step - loss: 0.1717 - f1_m: 0.9461 - val_loss: 2.2985 - val_f1_m: 0.0000e+00
Epoch 9/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1664 - f1_m: 0.9461 - val_loss: 2.2592 - val_f1_m: 0.0000e+00
Epoch 10/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1621 - f1_m: 0.9429 - val_loss: 2.2034 - val_f1_m: 0.0000e+00
0.9428879022598267
if any
Epoch 1/10
29/29 [==============================] - 0s 4ms/step - loss: 0.5229 - f1_m: 0.8384 - val_loss: 1.4357 - val_f1_m: 0.0000e+00
Epoch 2/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2915 - f1_m: 0.9461 - val_loss: 2.1078 - val_f1_m: 0.0000e+00
Epoch 3/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2243 - f1_m: 0.9461 - val_loss: 2.4856 - val_f1_m: 0.0000e+00
Epoch 4/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2063 - f1_m: 0.9461 - val_loss: 2.6594 - val_f1_m: 0.0000e+00
Epoch 5/10
29/29 [==============

29/29 [==============================] - 0s 1ms/step - loss: 0.1861 - f1_m: 0.9461 - val_loss: 2.4226 - val_f1_m: 0.0000e+00
Epoch 8/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1803 - f1_m: 0.9461 - val_loss: 2.3270 - val_f1_m: 0.0000e+00
Epoch 9/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1745 - f1_m: 0.9461 - val_loss: 2.2676 - val_f1_m: 0.0000e+00
Epoch 10/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1695 - f1_m: 0.9461 - val_loss: 2.1985 - val_f1_m: 0.0000e+00
0.9461206197738647
if any
Epoch 1/10
29/29 [==============================] - 0s 5ms/step - loss: 0.5217 - f1_m: 0.8513 - val_loss: 1.4235 - val_f1_m: 0.0000e+00
Epoch 2/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2919 - f1_m: 0.9429 - val_loss: 2.0959 - val_f1_m: 0.0000e+00
Epoch 3/10
29/29 [==============================] - 0s 2ms/step - loss: 0.2242 - f1_m: 0.9461 - val_loss: 2.3794 - val_f1_m: 0.0000e+00
Epoch 4/10
29/29 [==============

29/29 [==============================] - 0s 1ms/step - loss: 0.1929 - f1_m: 0.9429 - val_loss: 2.4447 - val_f1_m: 0.0000e+00
Epoch 7/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1859 - f1_m: 0.9461 - val_loss: 2.3399 - val_f1_m: 0.0000e+00
Epoch 8/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1797 - f1_m: 0.9429 - val_loss: 2.2984 - val_f1_m: 0.0000e+00
Epoch 9/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1747 - f1_m: 0.9461 - val_loss: 2.1862 - val_f1_m: 0.0000e+00
Epoch 10/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1698 - f1_m: 0.9461 - val_loss: 2.2002 - val_f1_m: 0.0000e+00
0.9461206793785095
if any
Epoch 1/10
29/29 [==============================] - 0s 4ms/step - loss: 0.5217 - f1_m: 0.8534 - val_loss: 1.4181 - val_f1_m: 0.0000e+00
Epoch 2/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2912 - f1_m: 0.9429 - val_loss: 2.1243 - val_f1_m: 0.0000e+00
Epoch 3/10
29/29 [==============

29/29 [==============================] - 0s 1ms/step - loss: 0.1935 - f1_m: 0.9461 - val_loss: 2.4676 - val_f1_m: 0.0000e+00
Epoch 6/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1833 - f1_m: 0.9429 - val_loss: 2.3968 - val_f1_m: 0.0000e+00
Epoch 7/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1742 - f1_m: 0.9429 - val_loss: 2.3377 - val_f1_m: 0.0000e+00
Epoch 8/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1667 - f1_m: 0.9461 - val_loss: 2.2673 - val_f1_m: 0.0000e+00
Epoch 9/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1604 - f1_m: 0.9397 - val_loss: 2.2294 - val_f1_m: 0.0000e+00
Epoch 10/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1548 - f1_m: 0.9397 - val_loss: 2.1531 - val_f1_m: 0.0000e+00
0.9396551847457886
if any
Epoch 1/10
29/29 [==============================] - 0s 4ms/step - loss: 0.5274 - f1_m: 0.8394 - val_loss: 1.4346 - val_f1_m: 0.0000e+00
Epoch 2/10
29/29 [==============

29/29 [==============================] - 0s 1ms/step - loss: 0.2099 - f1_m: 0.9461 - val_loss: 2.6033 - val_f1_m: 0.0000e+00
Epoch 5/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2008 - f1_m: 0.9429 - val_loss: 2.5800 - val_f1_m: 0.0000e+00
Epoch 6/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1934 - f1_m: 0.9461 - val_loss: 2.4859 - val_f1_m: 0.0000e+00
Epoch 7/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1865 - f1_m: 0.9461 - val_loss: 2.4167 - val_f1_m: 0.0000e+00
Epoch 8/10
29/29 [==============================] - 0s 2ms/step - loss: 0.1809 - f1_m: 0.9397 - val_loss: 2.3385 - val_f1_m: 0.0000e+00
Epoch 9/10
29/29 [==============================] - 0s 2ms/step - loss: 0.1756 - f1_m: 0.9461 - val_loss: 2.2052 - val_f1_m: 0.0000e+00
Epoch 10/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1710 - f1_m: 0.9429 - val_loss: 2.2175 - val_f1_m: 0.0000e+00
0.9428879022598267
if any
Epoch 1/10
29/29 [==============

29/29 [==============================] - 0s 1ms/step - loss: 0.2256 - f1_m: 0.9397 - val_loss: 2.4914 - val_f1_m: 0.0000e+00
Epoch 4/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2046 - f1_m: 0.9461 - val_loss: 2.5231 - val_f1_m: 0.0000e+00
Epoch 5/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1941 - f1_m: 0.9429 - val_loss: 2.5415 - val_f1_m: 0.0000e+00
Epoch 6/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1862 - f1_m: 0.9461 - val_loss: 2.4717 - val_f1_m: 0.0000e+00
Epoch 7/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1791 - f1_m: 0.9429 - val_loss: 2.4234 - val_f1_m: 0.0000e+00
Epoch 8/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1731 - f1_m: 0.9461 - val_loss: 2.2904 - val_f1_m: 0.0000e+00
Epoch 9/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1674 - f1_m: 0.9429 - val_loss: 2.3086 - val_f1_m: 0.0000e+00
Epoch 10/10
29/29 [==============================] - 0s 1ms

[[[array([[ 0.01112658,  0.00277015, -0.00862743, ...,  0.0008666 ,
           -0.01293376,  0.06385448],
          [ 0.07493433,  0.01825036, -0.00221062, ..., -0.07164883,
           -0.00725049,  0.08696105],
          [-0.03515964, -0.00209531,  0.03172271, ...,  0.08539461,
           -0.00580376, -0.02831915],
          ...,
          [ 0.00688852,  0.01664085,  0.0304131 , ..., -0.1317613 ,
            0.05459478, -0.03096615],
          [-0.02315127, -0.07718714,  0.11915966, ..., -0.11185814,
            0.09694653,  0.08861946],
          [ 0.01034362, -0.026957  , -0.01304596, ..., -0.01465187,
           -0.08493983, -0.07947924]], dtype=float32),
   array([-0.02764474, -0.01034944,  0.07108409, -0.02065446, -0.03215132,
           0.0645092 ,  0.06190839, -0.02137988,  0.0991112 ,  0.07994048],
         dtype=float32),
   array([[-0.6818601 ,  0.18101221],
          [-0.32493433,  0.21494554],
          [ 0.7052068 , -0.43321022],
          [-0.4729864 ,  0.50544655],
    

In [32]:
len(Models)

1

In [33]:
#here use only top 5-10 integrally private models.
#add_weights=add_weights[top_5]
A=np.argsort(np.array(Models).T[1])[::-1][:5]
print(np.array(Models).T[1])
print(A)
recommended_models=[]
for i in range(len(A)):
    recommended_models.append(add_weights[A[i]])
#add_weights=add_weights[A]

[50]
[0]


/Users/ayushkv/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ayushkv/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.


In [ ]:
# Now trying to generate Streaming settings for the dataset
# lets find the outputs from all the 
mean_model_weights, mean_model_acc, mean_model_loss, mean_model_test_acc, mean_model_test_loss = epsilon_mean_recommendation(recommended_models, data)

In [ ]:
# the issue is to check when there is actually a concept drift:- use synthetic data to generate concept drift
"""
#This wouild work more or less similar to drift_detection_each_model, removed for now
def drfit_detection_mean_models(mean_model_weights, incoming_data):
    prediction_probe=[]
    y_test=incoming_data[target_variable] 
    X_test=incoming_data.drop[target_variable]
    for i in range(len(mean_model_weights)):
        prediction_probe.append(mean_model_weights.predict_proba(X_test))
        #here onwards detect drift using  kswin or adwin
"""     

In [60]:
#right now trying for mean models
stream_mean_results=[]
current_window = data_init.copy()
delta=0.01
adwin = ADWIN(delta)
while stream.n_remaining_samples()>1:
    incoming_data = stream.next_sample(five_percent)
    #rint(incoming_data)
    data_incoming = pd.DataFrame(incoming_data[0], columns=data.columns[:-1])
    data_incoming[target_variable]=incoming_data[1]
    #print(data_incoming)
    detected, evaluate, adwin = drift_detection_each_model(recommended_models[0], data_incoming.copy(), adwin)
    stream_mean_results.append(evaluate)
    if detected:
        print("drift has been detecte models must be retrained")
        current_window = update_train_data(current_window, data_incoming)
        add_weights, Models, val_acc, train_acc, train_loss, val_acc, val_loss = get_IP_models(data_init, epsilon=0.05, target=2)
        A=np.argsort(np.array(Models).T[1])[::-1][:5]
        print(np.array(Models).T[1])
        print(A)
        recommended_models=[]
        for i in range(len(A)):
            recommended_models.append(add_weights[A[i]])
        #add_weights=add_weights[A]
        mean_model_weights, mean_model_acc, mean_model_loss, mean_model_test_acc, mean_model_test_loss = epsilon_mean_recommendation(recommended_models, data)
    else:
        print("One lap done.")
        continue

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [0. 1.]]
71/71 [==============================] - 0s 543us/step - loss: 0.5719 - accuracy: 0.7559
[0.5719394087791443, 0.7558602094650269]
71/71 [==============================] - 0s 485us/step - loss: 0.5763 - accuracy: 0.7559
[0.5763035416603088, 0.7558602094650269]
71/71 [==============================] - 0s 516us/step - loss: 0.5753 - accuracy: 0.7559
[0.5753345489501953, 0.7558602094650269]
71/71 [==============================] - 0s 532us/step - loss: 0.5795 - accuracy: 0.7559
[0.5795111656188965, 0.7558602094650269]
71/71 [==============================] - 0s 535us/step - loss: 0.5655 - accuracy: 0.7567
[0.5655084252357483, 0.7567448019981384]
71/71 [==============================] - 0s 566us/step - loss: 0.5893 - accuracy: 0.7559
[0.5892898440361023, 0.7558602094650269]
71/71 [==============================] - 0s 503us/step - loss: 0.5811 - accuracy: 0.7559
[0.5811470150947571, 0.7558602094650269]
71/71 [=======================

71/71 [==============================] - 0s 524us/step - loss: 0.5749 - accuracy: 0.7492
[0.5749021768569946, 0.7492260336875916]
71/71 [==============================] - 0s 466us/step - loss: 0.5805 - accuracy: 0.7492
[0.5804607272148132, 0.7492260336875916]
71/71 [==============================] - 0s 531us/step - loss: 0.5737 - accuracy: 0.7492
[0.5737321376800537, 0.7492260336875916]
71/71 [==============================] - 0s 485us/step - loss: 0.5829 - accuracy: 0.7492
[0.5828604698181152, 0.7492260336875916]
71/71 [==============================] - 0s 552us/step - loss: 0.5659 - accuracy: 0.7492
[0.5659412145614624, 0.7492260336875916]
71/71 [==============================] - 0s 482us/step - loss: 0.5886 - accuracy: 0.7492
[0.5885718464851379, 0.7492260336875916]
71/71 [==============================] - 0s 503us/step - loss: 0.5805 - accuracy: 0.7492
[0.5804657340049744, 0.7492260336875916]
71/71 [==============================] - 0s 497us/step - loss: 0.5761 - accuracy: 0.7492
[

71/71 [==============================] - 0s 502us/step - loss: 0.5434 - accuracy: 0.7643
[0.5433928370475769, 0.7642636299133301]
71/71 [==============================] - 0s 467us/step - loss: 0.5477 - accuracy: 0.7643
[0.5477036833763123, 0.7642636299133301]
71/71 [==============================] - 0s 465us/step - loss: 0.5409 - accuracy: 0.7643
[0.5409049987792969, 0.7642636299133301]
71/71 [==============================] - 0s 473us/step - loss: 0.5506 - accuracy: 0.7643
[0.5506163239479065, 0.7642636299133301]
71/71 [==============================] - 0s 468us/step - loss: 0.5270 - accuracy: 0.7651
[0.5270097851753235, 0.7651481628417969]
71/71 [==============================] - 0s 476us/step - loss: 0.5556 - accuracy: 0.7643
[0.5555615425109863, 0.7642636299133301]
71/71 [==============================] - 0s 467us/step - loss: 0.5476 - accuracy: 0.7643
[0.5476467609405518, 0.7642636299133301]
71/71 [==============================] - 0s 465us/step - loss: 0.5441 - accuracy: 0.7643
[

71/71 [==============================] - 0s 502us/step - loss: 0.6137 - accuracy: 0.7346
[0.6136763095855713, 0.7346307039260864]
71/71 [==============================] - 0s 464us/step - loss: 0.6155 - accuracy: 0.7346
[0.6154701113700867, 0.7346307039260864]
71/71 [==============================] - 0s 505us/step - loss: 0.6069 - accuracy: 0.7346
[0.6068885922431946, 0.7346307039260864]
71/71 [==============================] - 0s 477us/step - loss: 0.6162 - accuracy: 0.7346
[0.6161554455757141, 0.7346307039260864]
71/71 [==============================] - 0s 461us/step - loss: 0.5945 - accuracy: 0.7360
[0.5945320129394531, 0.7359575629234314]
71/71 [==============================] - 0s 469us/step - loss: 0.6223 - accuracy: 0.7346
[0.6223322153091431, 0.7346307039260864]
71/71 [==============================] - 0s 463us/step - loss: 0.6156 - accuracy: 0.7346
[0.6156183481216431, 0.7346307039260864]
71/71 [==============================] - 0s 463us/step - loss: 0.6119 - accuracy: 0.7346
[

71/71 [==============================] - 0s 501us/step - loss: 0.5390 - accuracy: 0.7678
[0.538967490196228, 0.7678018808364868]
71/71 [==============================] - 0s 466us/step - loss: 0.5417 - accuracy: 0.7678
[0.5417178869247437, 0.7678018808364868]
71/71 [==============================] - 0s 529us/step - loss: 0.5332 - accuracy: 0.7678
[0.5332463979721069, 0.7678018808364868]
71/71 [==============================] - 0s 475us/step - loss: 0.5412 - accuracy: 0.7678
[0.5411968231201172, 0.7678018808364868]
71/71 [==============================] - 0s 462us/step - loss: 0.5309 - accuracy: 0.7687
[0.5309058427810669, 0.7686864137649536]
71/71 [==============================] - 0s 476us/step - loss: 0.5426 - accuracy: 0.7678
[0.5426101088523865, 0.7678018808364868]
71/71 [==============================] - 0s 468us/step - loss: 0.5435 - accuracy: 0.7678
[0.5434769988059998, 0.7678018808364868]
71/71 [==============================] - 0s 485us/step - loss: 0.5380 - accuracy: 0.7678
[0

...

29/29 [==============================] - 0s 1ms/step - loss: 0.2295 - f1_m: 0.9429 - val_loss: 2.3431 - val_f1_m: 0.0000e+00
Epoch 8/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2182 - f1_m: 0.9397 - val_loss: 2.4436 - val_f1_m: 0.0000e+00
Epoch 9/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2123 - f1_m: 0.9461 - val_loss: 2.5303 - val_f1_m: 0.0000e+00
Epoch 10/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2078 - f1_m: 0.9364 - val_loss: 2.5893 - val_f1_m: 0.0000e+00
0.9364224076271057
if any
Epoch 1/10
29/29 [==============================] - 0s 4ms/step - loss: 0.6766 - f1_m: 0.7241 - val_loss: 0.7749 - val_f1_m: 0.0000e+00
Epoch 2/10
29/29 [==============================] - 0s 1ms/step - loss: 0.5747 - f1_m: 0.9461 - val_loss: 0.9538 - val_f1_m: 0.0000e+00
Epoch 3/10
29/29 [==============================] - 0s 1ms/step - loss: 0.4600 - f1_m: 0.9429 - val_loss: 1.2201 - val_f1_m: 0.0000e+00
Epoch 4/10
29/29 [==============

29/29 [==============================] - 0s 1ms/step - loss: 0.2849 - f1_m: 0.9397 - val_loss: 1.8600 - val_f1_m: 0.0000e+00
Epoch 6/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2447 - f1_m: 0.9461 - val_loss: 2.0871 - val_f1_m: 0.0000e+00
Epoch 7/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2223 - f1_m: 0.9429 - val_loss: 2.2375 - val_f1_m: 0.0000e+00
Epoch 8/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2079 - f1_m: 0.9364 - val_loss: 2.2934 - val_f1_m: 0.0000e+00
Epoch 9/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1978 - f1_m: 0.9461 - val_loss: 2.2245 - val_f1_m: 0.0000e+00
Epoch 10/10
29/29 [==============================] - 0s 1ms/step - loss: 0.1895 - f1_m: 0.9461 - val_loss: 2.2620 - val_f1_m: 0.0000e+00
0.9461206197738647
if any
Epoch 1/10
29/29 [==============================] - 0s 4ms/step - loss: 0.6768 - f1_m: 0.7274 - val_loss: 0.7751 - val_f1_m: 0.0000e+00
Epoch 2/10
29/29 [==============

29/29 [==============================] - 0s 1ms/step - loss: 0.4614 - f1_m: 0.9461 - val_loss: 1.2140 - val_f1_m: 0.0000e+00
Epoch 4/10
29/29 [==============================] - 0s 1ms/step - loss: 0.3580 - f1_m: 0.9461 - val_loss: 1.5359 - val_f1_m: 0.0000e+00
Epoch 5/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2872 - f1_m: 0.9461 - val_loss: 1.8631 - val_f1_m: 0.0000e+00
Epoch 6/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2474 - f1_m: 0.9461 - val_loss: 2.1384 - val_f1_m: 0.0000e+00
Epoch 7/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2259 - f1_m: 0.9397 - val_loss: 2.3082 - val_f1_m: 0.0000e+00
Epoch 8/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2155 - f1_m: 0.9429 - val_loss: 2.4167 - val_f1_m: 0.0000e+00
Epoch 9/10
29/29 [==============================] - 0s 1ms/step - loss: 0.2086 - f1_m: 0.9461 - val_loss: 2.4752 - val_f1_m: 0.0000e+00
Epoch 10/10
29/29 [==============================] - 0s 1ms

/Users/ayushkv/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/Users/ayushkv/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


424/424 [==============================] - 0s 452us/step - loss: 0.6719 - accuracy: 0.7519
